In [1]:
import pandas as pd
import numpy as np

In [2]:
print("--- Step 1: Loading Datasets ---")

# Load df_analysis.csv as a plain DataFrame.
try:
    df_analysis = pd.read_csv('..\data\df_analysis.csv')
    print("Successfully loaded df_analysis.csv")
    print(f"df_analysis initial shape: {df_analysis.shape}")
except FileNotFoundError:
    print("ERROR: df_analysis.csv not found. Please ensure it's in the same directory.")
    # Stop execution if file is not found
    exit()

# Load RH_KTM.csv, parsing the 'Time' column and setting it as the index.
try:
    df_rh = pd.read_csv('..\data\RH_KTM.csv', parse_dates=['Time'], index_col='Time')
    print("Successfully loaded RH_KTM.csv")
    print(f"Original RH data shape: {df_rh.shape}")
except FileNotFoundError:
    print("ERROR: RH_KTM.csv not found. Please ensure it's in the same directory.")
    # Stop execution if file is not found
    exit()

--- Step 1: Loading Datasets ---
Successfully loaded df_analysis.csv
df_analysis initial shape: (9422, 12)
Successfully loaded RH_KTM.csv
Original RH data shape: (30180, 1)


In [3]:
df_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9422 entries, 0 to 9421
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Precipitation_mm              9422 non-null   float64
 1   Min_Temp_C                    9422 non-null   float64
 2   Max_Temp_C                    9422 non-null   float64
 3   Precipitation_7d_rolling_sum  9422 non-null   float64
 4   Year                          9422 non-null   int64  
 5   Month                         9422 non-null   int64  
 6   Day_of_Year                   9422 non-null   int64  
 7   Day_of_Week                   9422 non-null   int64  
 8   Month_sin                     9422 non-null   float64
 9   Month_cos                     9422 non-null   float64
 10  Day_of_Year_sin               9422 non-null   float64
 11  Day_of_Year_cos               9422 non-null   float64
dtypes: float64(8), int64(4)
memory usage: 883.4 KB


In [4]:
df_analysis.index

RangeIndex(start=0, stop=9422, step=1)

In [5]:
print("\n--- Step 2: Prepare DataFrames for Joining ---")

# 2a. Reconstruct DatetimeIndex for df_analysis
print("Reconstructing date index for df_analysis...")
# Use pd.to_datetime with the specified format '%Y-%j' for Year and Day_of_Year
# This is a robust way to handle dates without needing a 'Day' or 'Month' column directly
df_analysis['Date'] = pd.to_datetime(df_analysis['Year'].astype(str) + '-' + df_analysis['Day_of_Year'].astype(str), format='%Y-%j')
df_analysis.set_index('Date', inplace=True)
print("-> df_analysis index set successfully.")


--- Step 2: Prepare DataFrames for Joining ---
Reconstructing date index for df_analysis...
-> df_analysis index set successfully.


In [6]:
df_analysis.head()

,Precipitation_mm,Min_Temp_C,Max_Temp_C,Precipitation_7d_rolling_sum,Year,Month,Day_of_Year,Day_of_Week,Month_sin,Month_cos,Day_of_Year_sin,Day_of_Year_cos
Date,,,,,,,,,,,,
1999-01-01,0.0,0.3,20.2,0.0,1999,1,1,4,0.5,0.866025,0.017202,0.999852
1999-01-02,0.0,0.8,21.0,0.0,1999,1,2,5,0.5,0.866025,0.034398,0.999408
1999-01-03,0.0,0.5,23.6,0.0,1999,1,3,6,0.5,0.866025,0.051584,0.998669
1999-01-04,0.0,1.2,22.2,0.0,1999,1,4,0,0.5,0.866025,0.068755,0.997634
1999-01-05,0.0,1.0,21.7,0.0,1999,1,5,1,0.5,0.866025,0.085906,0.996303


In [7]:
# 2b. Process Relative Humidity Data
print("Processing and resampling RH data...")
# Rename the column for clarity
df_rh.rename(columns={'Kathmandu Airport': 'RH'}, inplace=True)
# Resample sub-daily data to daily frequency, calculating key statistics
rh_daily_agg = {'RH': ['mean', 'min', 'max']}
df_rh_daily = df_rh.resample('D').agg(rh_daily_agg)
# Flatten the multi-level columns
df_rh_daily.columns = ['RH_mean', 'RH_min', 'RH_max']
print("-> RH data resampled to daily frequency.")

Processing and resampling RH data...
-> RH data resampled to daily frequency.


In [14]:
df_rh.describe()

,RH
count,30176.000000
mean,77.350507
std,16.797110
min,-26.200000
25%,67.100000
50%,80.500000
75%,91.000000
max,151.100000


In [13]:
print("\n--- Step 3: Joining df_analysis with Daily RH Data ---")

# Now that both dataframes have a proper DatetimeIndex, we can join them.
df_historical = df_analysis.join(df_rh_daily)
print(f"Shape after join: {df_historical.shape}")
print("Columns added:", df_rh_daily.columns.to_list())
df_historical.describe().T


--- Step 3: Joining df_analysis with Daily RH Data ---
Shape after join: (9422, 15)
Columns added: ['RH_mean', 'RH_min', 'RH_max']


,count,mean,std,min,25%,50%,75%,max
Precipitation_mm,9422.0,4.304252,11.174712,0.000000,0.000000,0.000000e+00,2.500000,239.700000
Min_Temp_C,9422.0,12.999331,6.427786,-3.000000,7.400000,1.350000e+01,19.400000,23.000000
Max_Temp_C,9422.0,26.158098,4.384200,8.100000,23.000000,2.730000e+01,29.600000,35.600000
Precipitation_7d_rolling_sum,9422.0,30.129762,43.723155,0.000000,0.000000,8.200000e+00,45.700000,310.500000
Year,9422.0,2011.400870,7.446878,1999.000000,2005.000000,2.011000e+03,2018.000000,2024.000000
Month,9422.0,6.485247,3.436281,1.000000,4.000000,6.000000e+00,9.000000,12.000000
Day_of_Year,9422.0,181.973785,105.041493,1.000000,91.000000,1.820000e+02,272.000000,366.000000
Day_of_Week,9422.0,3.000000,2.000106,0.000000,1.000000,3.000000e+00,5.000000,6.000000
Month_sin,9422.0,-0.001898,0.707273,-1.000000,-0.866025,1.224647e-16,0.500000,1.000000
Month_cos,9422.0,-0.008809,0.706958,-1.000000,-0.866025,-1.836970e-16,0.500000,1.000000


In [10]:
print("\n--- Step 4: Final Cleaning and Saving ---")

# Check for missing values that may have been introduced by the join
print("Missing values per column after merge:")
print(df_historical.isnull().sum())


--- Step 4: Final Cleaning and Saving ---
Missing values per column after merge:
Precipitation_mm                0
Min_Temp_C                      0
Max_Temp_C                      0
Precipitation_7d_rolling_sum    0
Year                            0
Month                           0
Day_of_Year                     0
Day_of_Week                     0
Month_sin                       0
Month_cos                       0
Day_of_Year_sin                 0
Day_of_Year_cos                 0
RH_mean                         1
RH_min                          1
RH_max                          1
dtype: int64


In [11]:
# Use time-based interpolation to fill any small gaps
print("\nInterpolating missing values...")
df_historical.interpolate(method='time', inplace=True)

# Drop any rows with remaining NaNs (likely at the very start or end)
df_historical.dropna(inplace=True)
print("Dropped any remaining NaN rows.")

print(f"Final check: {df_historical.isnull().sum().sum()} total missing values remaining.")


Interpolating missing values...
Dropped any remaining NaN rows.
Final check: 0 total missing values remaining.


In [12]:
# Save the final, clean dataframe to a new CSV file.
df_historical.to_csv('..\data\df_historical.csv')

print(f"\n✅ Success! The historical dataset has been created and saved as 'df_historical.csv'")
print(f"Final shape of df_historical: {df_historical.shape}")
print("--- Final DataFrame Head ---")
print(df_historical.head())


✅ Success! The historical dataset has been created and saved as 'df_historical.csv'
Final shape of df_historical: (9422, 15)
--- Final DataFrame Head ---
            Precipitation_mm  Min_Temp_C  Max_Temp_C  \
Date                                                   
1999-01-01               0.0         0.3        20.2   
1999-01-02               0.0         0.8        21.0   
1999-01-03               0.0         0.5        23.6   
1999-01-04               0.0         1.2        22.2   
1999-01-05               0.0         1.0        21.7   

            Precipitation_7d_rolling_sum  Year  Month  Day_of_Year  \
Date                                                                 
1999-01-01                           0.0  1999      1            1   
1999-01-02                           0.0  1999      1            2   
1999-01-03                           0.0  1999      1            3   
1999-01-04                           0.0  1999      1            4   
1999-01-05                      